# 1. Load the python packages

In [ ]:
import pkg_resources
import pip
installedPackages = {pkg.key for pkg in pkg_resources.working_set}
required = {'nltk', 'spacy', 'textblob', 'backtrader'}
missing = required - installedPackages
if missing:
    !pip install nltk==3.4
    !pip install textblob==0.15.3
    !pip install -U SpaCy==2.2.0
    !python -m spacy download en_core_web_lg
    !pip install backtrader==1.9.74.123
!pip install tensorflow

In [ ]:
#NLP libraries
from textblob import TextBlob
import spacy
import nltk
import warnings
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import csv
import pandas as pd

#Run the command python -m spacy download en_core_web_sm to download this
#https://spacy.io/models
import en_core_web_lg
nlp = en_core_web_lg.load()

#Libraries for processing the news headlines
from lxml import etree
import json
from io import StringIO
from os import listdir
from os.path import isfile, join
from pandas.tseries.offsets import BDay
from scipy.stats.mstats import winsorize
from copy import copy

# Libraries for Classification for modeling the sentiments
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Keras package for the deep learning model for the sentiment prediction.
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation
from tensorflow.keras.layers import Embedding

# Load libraries
import statsmodels.api as sm
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
from datetime import date
import matplotlib.pyplot as plt
import yfinance as yf

#Additional Libraries
import json
import zipfile
import os.path
import sys

#Diable the warnings
import warnings
warnings.filterwarnings('ignore')

# 2. Load dataset

In [ ]:
# tickers = ['AAPL','MSFT','AMZN','GOOG','FB','WMT','JPM','TSLA','NFLX','ADBE']
# start = '2010-01-01'
# end = '2018-12-31'
# df_ticker_return = pd.DataFrame()
# for ticker in tickers:
#     ticker_yf = yf.Ticker(ticker)
#     if df_ticker_return.empty:
#         df_ticker_return = ticker_yf.history(start = start, end = end)
#         df_ticker_return['ticker']= ticker
#     else:
#         data_temp = ticker_yf.history(start = start, end = end)
#         data_temp['ticker']= ticker
#         df_ticker_return = df_ticker_return.append(data_temp)
# df_ticker_return.to_csv(r'Data\Step2.2_ReturnData.csv')

In [ ]:
from google.colab import files
uploaded = files.upload()

# load Step2.2_ReturnData.csv # not needed unless engineer indicators
# load Raw Headline Data.zip # not needed
# load Step3_NewsAndReturnData.csv # not needed unless compute new sentiment score
# load Step4_DataWithSentimentsResults.csv

# 3. Build the trading strategy

In [ ]:
# Define your tickers list
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'WMT', 'JPM', 'TSLA', 'NFLX', 'ADBE']

# Load the data
data_df = pd.read_csv(r'Step4_DataWithSentimentsResults.csv', sep='|')

# Replace 'FB' with 'META' in the 'ticker' column
data_df['ticker'] = data_df['ticker'].replace('FB', 'META')

# Filter the data to include only rows with the specified tickers
data_df = data_df[data_df['ticker'].isin(tickers)]

In [ ]:
###########################
# Setting up the strategy #
###########################

import backtrader as bt
import backtrader.indicators as btind
import backtrader.analyzers as btanalyzers

class Sentiment(bt.Indicator):
    lines = ('sentiment',)
    plotinfo = dict(
        plotymargin=0.5,
        plothlines=[0],
        plotyticks=[1.0, 0, -1.0])

    def next(self):
        self.sentiment = 0.0
        self.date = self.data.datetime
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]
        self.lines.sentiment[0] = self.sentiment

class SentimentStrat(bt.Strategy):
    params = (
        ('period', 15),
        ('printlog', True),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        # Keep track of pending orders
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.period)
        self.date = self.data.datetime
        self.sentiment = None
        Sentiment(self.data)
        self.plotinfo.plot = False
        self.buy_signals = []  # List to store buy points
        self.sell_signals = []  # List to store sell points

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    ### Main Strat ###
    def next(self):
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]

        # Check if an order is pending. if yes, we cannot send a 2nd one
        if self.order:
            return
        # If not in the market and previous sentiment not none
        if not self.position and prev_sentiment:
            # buy if current close more than sma AND sentiment increased by >= 0.5
            if self.dataclose[0] > self.sma[0] and self.sentiment - prev_sentiment >= 0.5:
                self.log('Previous Sentiment %.2f, New Sentiment %.2f BUY CREATE, %.2f' % (prev_sentiment, self.sentiment, self.dataclose[0]))
                self.order = self.buy()
                self.buy_signals.append((self.data.datetime.date(0), self.data.close[0]))  # Record buy signal

        # Already in the market and previous sentiment not none
        elif prev_sentiment:
            # sell if current close less than sma AND sentiment decreased by >= 0.5
            if self.dataclose[0] < self.sma[0] and self.sentiment - prev_sentiment <= -0.5:
                self.log('Previous Sentiment %.2f, New Sentiment %.2f SELL CREATE, %.2f' % (prev_sentiment, self.sentiment, self.dataclose[0]))
                self.order = self.sell()
                self.sell_signals.append((self.data.datetime.date(0), self.data.close[0]))  # Record sell signal

    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.period, self.broker.getvalue()), doprint=True)

#############################################
# Function for running the trading strategy #
#############################################

def run_strategy(ticker, start, end):
    print(ticker)
    ticker_data = yf.Ticker(ticker)
    df_ticker = ticker_data.history(start=start, end=end)

    # Initialize Cerebro and strategy
    cerebro = bt.Cerebro()
    sentiment_strategy = SentimentStrat  # Assuming your strategy is called SentimentStrat
    cerebro.addstrategy(sentiment_strategy)

    # Load data into Cerebro
    data = bt.feeds.PandasData(dataname=df_ticker)
    cerebro.adddata(data)

    # Set initial cash and other parameters
    initial_cash = 100000.0
    cerebro.broker.setcash(initial_cash)
    cerebro.addsizer(bt.sizers.FixedSize, stake=100)
    print(f'Starting Portfolio Value: {initial_cash:.2f}')

    # Run the strategy
    strat_instance = cerebro.run()[0]  # Get the strategy instance

    # Extract the dates and closing prices
    dates = df_ticker.index
    close_prices = df_ticker['Close']

    # Extract buy/sell signals from the strategy instance
    buy_signals = strat_instance.buy_signals
    sell_signals = strat_instance.sell_signals

    # Plot the closing price and buy/sell markers
    plt.figure(figsize=(12, 6))
    plt.plot(dates, close_prices, label=f"{ticker} Closing Price", color="blue")

    # Plot buy and sell signals
    if buy_signals:
        buy_dates, buy_prices = zip(*buy_signals)
        plt.scatter(buy_dates, buy_prices, marker='^', color='green', label='Buy Signal', s=100)
    if sell_signals:
        sell_dates, sell_prices = zip(*sell_signals)
        plt.scatter(sell_dates, sell_prices, marker='v', color='red', label='Sell Signal', s=100)

    # Customize and display the plot
    plt.title(f"{ticker} Strategy Performance with Buy/Sell Signals")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.show()

    # Display portfolio summary
    final_cash = cerebro.broker.getvalue()
    print(f'Start Portfolio value: {initial_cash:.2f}\nFinal Portfolio Value: {final_cash:.2f}\nProfit: {final_cash - initial_cash:.2f}')

    return float(close_prices[0]), (final_cash - initial_cash)

## Results for Individual Stocks

In [ ]:
ticker = 'GOOG'
date_sentiment=data_df[data_df['ticker'].isin([ticker])]
date_sentiment=date_sentiment[['date','sentiment_lex']]
date_sentiment['date']=pd.to_datetime(date_sentiment['date'], format='%Y-%m-%d').dt.date
date_sentiment=date_sentiment.set_index('date')['sentiment_lex']
date_sentiment=date_sentiment.to_dict()
run_strategy(ticker, start = '2012-01-01', end = '2018-12-12')

## Results for Multiple Stocks

In [ ]:
results_tickers = {}
for ticker in tickers:
    date_sentiment=data_df[data_df['ticker'].isin([ticker])]
    date_sentiment=date_sentiment[['date','sentiment_lex']]
    date_sentiment['date']=pd.to_datetime(date_sentiment['date'], format='%Y-%m-%d').dt.date
    date_sentiment=date_sentiment.set_index('date')['sentiment_lex']
    date_sentiment=date_sentiment.to_dict()
    results_tickers[ticker] = run_strategy(ticker, start = '2012-01-01', end = '2018-12-12')

In [ ]:
df_profit_all = pd.DataFrame.from_dict(results_tickers).set_index([pd.Index(["PerUnitStartPrice", 'StrategyProfit'])])

df_profit_all

In [ ]:
strategy_profit_sum = df_profit_all.loc["StrategyProfit"].sum()
print("Total Strategy Profit:", strategy_profit_sum)

In [ ]:
results_tickers = {}
for ticker in tickers:
    date_sentiment=data_df[data_df['ticker'].isin([ticker])]
    date_sentiment=date_sentiment[['date','sentiment_lex']]
    date_sentiment['date']=pd.to_datetime(date_sentiment['date'], format='%Y-%m-%d').dt.date
    date_sentiment=date_sentiment.set_index('date')['sentiment_lex']
    date_sentiment=date_sentiment.to_dict()
    results_tickers[ticker] = run_strategy(ticker, start = '2016-01-01', end = '2018-12-12')

In [ ]:
df_profit_all = pd.DataFrame.from_dict(results_tickers).set_index([pd.Index(["PerUnitStartPrice", 'StrategyProfit'])])

df_profit_all

## Varying the Strategy Time Period for Individual Stocks

In [ ]:
import random
import pandas as pd
from datetime import timedelta

# Assuming your data is loaded as 'data_df' and spans '2012-01-01' to '2018-12-12'
ticker = 'GOOG'
start_date = pd.to_datetime('2012-01-01')
end_date = pd.to_datetime('2018-12-12')
total_days = (end_date - start_date).days

# Set seed for reproducibility
random.seed(42)

# Generate 100 random start dates within the allowable range
period_length = 365*2
num_period = 100
periods = [
    start_date + timedelta(days=day_offset)
    for day_offset in random.sample(range(total_days - period_length), num_period)
]

# Calculate and store profit for each period
profits = []
for start in periods:
    period_start = start.strftime('%Y-%m-%d')
    period_end = (start + timedelta(days=period_length)).strftime('%Y-%m-%d')
    _, profit = run_strategy(ticker, start=period_start, end=period_end)
    profits.append((period_start, period_end, profit))

# Convert to a DataFrame for analysis
profit_df = pd.DataFrame(profits, columns=["Period Start", "Period End", "Profit"])
print(profit_df)

In [ ]:
# Display descriptive statistics for the 'Profit' column
profit_stats = profit_df['Profit'].describe()
print(profit_stats)

## Varying the Strategy Time Period for Multiple Stocks

In [ ]:
import random
import pandas as pd
from datetime import timedelta

# Assuming your data is loaded as 'data_df' and spans '2012-01-01' to '2018-12-12'
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'WMT', 'JPM', 'TSLA', 'NFLX', 'ADBE']
start_date = pd.to_datetime('2012-01-01')
end_date = pd.to_datetime('2018-12-12')
total_days = (end_date - start_date).days

# Set seed for reproducibility
random.seed(42)

# Define period length and number of periods
period_length = 365*2  # Length of each period in days
num_period = 100    # Number of periods

# Dictionary to store descriptive stats for each ticker
descriptive_stats = {}

# Iterate over each ticker
for ticker in tickers:
    # Generate 100 random start dates within the allowable range
    periods = [
        start_date + timedelta(days=day_offset)
        for day_offset in random.sample(range(total_days - period_length), num_period)
    ]

    # Calculate and store profit for each period
    profits = []
    for start in periods:
        period_start = start.strftime('%Y-%m-%d')
        period_end = (start + timedelta(days=period_length)).strftime('%Y-%m-%d')
        _, profit = run_strategy(ticker, start=period_start, end=period_end)
        profits.append(profit)

    # Calculate descriptive statistics for the ticker
    profit_series = pd.Series(profits)
    stats = profit_series.describe()

    # Add ticker stats to dictionary
    descriptive_stats[ticker] = stats

# Convert the dictionary to a DataFrame
descriptive_stats_df = pd.DataFrame(descriptive_stats)

In [ ]:
descriptive_stats_df